**Import packages**

In [0]:
import sys
sys.path.append('dbfs:/FileStore/tables/')

from schema_utils import validate_schema
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

**Define paths**

In [0]:
bronze_path = "abfss://bronze@stfinancedev.dfs.core.windows.net/bnz_branches"
silver_path = "abfss://silver@stfinancedev.dfs.core.windows.net/s_branches"

### **Schema Validation**

In [0]:
#Define schema
expected_schema = StructType([
    StructField('BranchID', StringType(), True), 
    StructField('BankCode', StringType(), True), 
    StructField('BranchName', StringType(), True), 
    StructField('Address', StringType(), True), 
    StructField('City', StringType(), True), 
    StructField('State', StringType(), True), 
    StructField('Pincode', StringType(), True), 
    StructField('IFSC', StringType(), True), 
    StructField('ManagerName', StringType(), True), 
    StructField('CreatedAt', StringType(), True)   
])

In [0]:
#load incoming data
incoming_df = spark.read.format("parquet").load(bronze_path)

In [0]:
#schema validation
validate_schema(incoming_df, expected_schema,"Branches")

In [0]:
incoming_df.display()

### **Data validations**

In [0]:
#null handle
df_clean = incoming_df.na.drop(subset=['BranchID','BankCode','BranchName','Address','City','State','Pincode','IFSC','ManagerName','CreatedAt'])

#duplicate handle 
df_clean = df_clean.dropDuplicates(['BranchID','BankCode','BranchName','Address','City','State','Pincode','IFSC','ManagerName','CreatedAt'])



### **Write in silver layer**

In [0]:
df_clean.write.format("delta")\
              .mode("overwrite")\
              .option("mergeSchema", True)\
              .partitionBy("State")\
              .save(silver_path)